In [1]:
import pandas as pd
import numpy as np
from geopy.distance import geodesic
import ast

In [3]:
!pip install chardet
!pip install scipy



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.3/30.3 MB 32.6 MB/s eta 0:00:00a 0:00:01

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [4]:
import chardet

# Detect the file encoding
with open("your_local_path/Global Open Litter Data - UK.csv", 'rb') as f:
    result = chardet.detect(f.read())
    encoding = result['encoding']

# Read the CSV with the detected encoding
df = pd.read_csv("your_local_path/Global Open Litter Data - UK.csv", encoding=encoding)


In [13]:
# Step 2: Remove rows where userTags is an empty list (i.e., '[]')
# Ensure userTags is treated as a string and drop rows where it's empty or null
df = df[df["userTags"].notnull()]  # Remove rows where userTags is NaN
df = df[df["userTags"].str.strip() != ""]  # Remove rows where userTags is empty or whitespace

df = df[df["userTags"].apply(lambda x: x != '[]')]

In [14]:


# Step 3: Define a function to categorize userTags
def categorize_user_tags(tags):
    if isinstance(tags, str):  # Ensure tags is a string
        try:

            # Convert semicolon-separated string into a list of tags
            tags_list = [tag.strip().lower() for tag in tags.split(";") if tag.strip()]  # Split by ";", strip spaces, and lowercase
            tags_set = set(tags_list)  # Convert to set for faster lookup

            # # Safely convert the string to a list
            # tags_list = ast.literal_eval(tags.lower())  
            # if not isinstance(tags_list, list):  # Ensure it's a list
            #     return "Uncategorized"
            
            # tags_set = {tag.strip() for tag in tags_list}  # Remove extra spaces

            # Define the categories and associated keywords
            categories = {
            "Plastic Litter": {"plsstic", "coffeecuplid", "balloon", "flosser", "knife", "flag", "ball", "cleaningcloth", "pen","handle", "giftcard", "tennisball", "label", "tubing", "tag", "toy", "rudder", "ketchup", "wiffleball", "softball", "coffeecup", "pencil", "brush", "bottle", "wrapper", "plastic", "bag", "doritochipbag", "straw", "filter", "whopperwrapper", "plasticbag", "snickerwrapper", "ketchuppacket", "chipbag", "facemask", "styrofoam", "foam", "polystyrene", "waterbottle", "plasticstraw", "garbage", "beerbottle", "plasticpeukmeuk2020", "wrappers", "chipsbag", 
        "chocolatewrapper", "plasticdogpoobag", "plasticunidentifiedobject", "Garbage", "canwrapper", "cigarillowrapper", "candywrapper", "plasticcup", "plasticpackaging", "paintbottle", 
        "cigarwrapper", "glassbottle", "plastics", "dogbag", "potatochipsbag", "cellophanewrapper", 
        "plasticcasing", "dogpoopbag", "yogurtcup(plastic)", "krogerbottledwaterlabel", "twixwrapper", 
        "doritoschipbag", "waterbottlewrapper", "trashbag", "plasticcup", "foodwrapper", "styrofoamcup", 
        "plasticfilm", "plastichanger", "chipsbag", "paperwrapper", "strawwrapper", "candywrapper", "plasticbottle",
        "blackplastic", "brokenbottlecap", "bandaidwrapper", "dogpoobag", "plasticcontainer", "paperwrapper", 
        "herseyswrapper", "granolabarwrapper", "pieceofbag", "paperbag", "ziplockbag", "bottlering", "carrierbag",
        "gumwrapper", "plasticlid", "goldfishwrapper", "ricekrispywrapper", "plasticbox", "spraybottle", "styrafoam",
        "plasticcup", "stryofoam", "plasticballgreen", "Garbage", "bottlecap", "plasticwrapper", "gumwrapper", 
        "snackbag", "bluntwrapper", "trashbag", "plasticlid", "plasticcigarwrapper", "burgerwrapper", 
        "plasticwatterbottle", "plasticstraw", "stryfoam", "stryofoam", "cigarillowrapper", "straws", "grocerybag", 
        "plasticbottle", "stryfoam", "vodkabottle", "snackbag", "plasticball", "stryrofoam", "binding", "drink", "cup", "spoon", "ringpull", "mask", "nonwoven", "lid", "sponge", "disposable", "sox", "papet", "lacrosseball", "cilp", "fome", "tape", "plasticstraw", "plasticwrap", "plasticpackaging", "plasticcup", "trashbag", "plasticwrapper", "plasticbottle", "plasticfilm", "plasticbox", "plasticball", "plasticcasing", "plasticcontainer"},
            "Metal Litter": {"wire", "firework", "screw", "keychain", "aluminum", "foils", "can", "metal", "cap", "scrap", "canwrapper", "bottlecap", "candywrapper", "aluminumcan", "candywrapper", "landscapetape", "candy", 
        "candywrapper", "brokenbottlecap", "candybox", "candy", "sodacan", "beercan", "redbullcan", "aluminumfoil", 
        "candleholder", "bottlecap", "cans", "caprisun", "aluminumfoil", "beercan", "aluminum", "breadclip", "goldfish", "sealant", "exposingtrash", "alumminum", "gaderadewraper", "mylarballoon", "tinfoil", "soda", "cans", "beercan", "aluminumfoil", "redbullcan", "sodacan"},
            "Glass Litter": {"saucecup","glass", "jar", "bottle", "broken", "beerbottle", "bottlecap", "paintbottle", "glassbottle", "krogerbottledwaterlabel", "waterbottlewrapper", 
        "plasticbottle", "brokenbottlecap", "bottlering", "spraybottle", "brokenglass", "bottlecap", "eyeglasses", 
        "plasticwatterbottle", "plasticbottle", "vodkabottle", "glassbottle", "paintbottle", "glass", "brokenbottle", "mug"},
            "Paper and Cardboard Litter": {"box", "nappkin", "ticket", "magazine", "napkin", "tissue", "giftcardholder", "trashtag", "reciept", "paper", "cardboard", "newspaper", "carton", "flyer", "packaging", "foilpaper", "drinkcarton", "toiletpaper", "paperreceipt", "papernapkin", "plasticpackaging", "chewinggumpackaging",
        "drinkcarton", "paperwrapper", "papercarton", "paperwrapper", "papertowel", "paperbag", "paperbox", "papercup", 
        "cardboardroll", "paperplate", "vapepackaging", "ccardboard", "cardboardbox", "paperbox", "orangejuicecarton", 
        "papercup", "papertowels", "papertissue", "receipt", "papercup", "paperbag", "paperplate", "paperwrapper", "paperbox", "cardboardroll", "paperbox", "cellophane", "paperreceipt", "papercarton"},
            "Organic and Food Litter": {"bacon", "cheese", "burger", "plant", "kitkat", "bannana", "gum", "coffee", "drink", "m&m", "orea", "milk", "coconut", "food", "scrap", "grass", "leaves", "organic", "foodwaste", "foodwrapper", "foodwaste", "foodcup", "foodwaste", "compostable", "gardening", "foodwrapper"},
            "Cigarette Litter": {"cigarillo", "cigar", "cigarette", "butt", "filter", "pack", "cigarettebutt", "cirgarettebutts", "cigarettepack", "cigarettebutts", "cigarettepack", "packingpeanut", "cigarettes", "packet", 
        "plasticpackaging", "condimentpacket", "saucepacket", "newportcigarettebox", "chewinggumpackaging", "packet", 
        "packing", "cigarettepack", "package", "hotsaucepacket", "vapepackaging", "cirgaratpackets", "packet", "cigarettebutts", "cigarettepack", "cigarette", "cigar"},
            "E-Waste (Electronic Waste)": {"electronics", "airpod", "battery", "phone", "appliance", "cable", "e-waste", "cellophane"},
            "Hazardous Waste": {"tape", "dryersheet", "bandage", "cream", "chemical", "paint", "pesticide", "medical", "concrete", "paintbottle", "paintbottle", "chemical", "toxic", "pesticide", "medical"},
            "Rubber and Tire Litter": {"tire", "rubber", "footwear", "rubberband", "rubberball", "rubberglove", "rubberband", "rubberball", "tire", "rubberglove"},
            "Marine Litter": {"fishing", "net", "marine", "rope", "nylonrope", "netal", "fishinggear", "netting"},
            "Cloth Litter": {"cotton", "shoelace", "jacket", "towel", "sandal", "shoe", "clothes", "leather", "blanket","carpart", "hat", "shorts", "glove", "shirt", "clothing", "fabric", "textile", "sweater", "clothingtag", "rubberglove", "Gloves", "latexglove", "cloth"}
        }

            # Check which category the tags match
            for category, keywords in categories.items():
                if tags_set & keywords:  # Check for overlap
                    return category

        except (ValueError, SyntaxError):  # Catch parsing errors
            return "Uncategorized"

    return "Uncategorized"  # Default if no match or invalid input

In [15]:
# Step 4: Apply the function to the userTags column
df["category"] = df["userTags"].apply(categorize_user_tags)

# Step 5: Save or display the updated dataset
# Save the updated dataset to a new file
df.to_csv("england_global_open_litter_data_material_category.csv", index=False)

# Print the updated DataFrame (optional)
print(df)

            OID_  litterId       long        lat           timestamp  \
0         793967   2480016   5.246294  52.391107  2/10/2019 13:14:07   
1       11076645    577848 -81.538590  26.278606   5/27/2017 8:25:17   
2       11076646    577850 -81.538590  26.278606   5/27/2017 8:25:17   
3       15542639     88350  -5.056561  50.426128   1/1/2015 20:03:08   
5       15542641     88608  -2.193209  52.528403   1/3/2015 10:45:04   
...          ...       ...        ...        ...                 ...   
682649  18908793   1093363  51.495582  -2.545121   2/24/2018 0:30:38   
682650  18908911   1139003  51.495736  -2.544095   3/22/2018 0:34:33   
682651  18908912   1139004  51.495776  -2.544435   3/22/2018 0:35:06   
682652  18908913   1139006  51.495946  -2.544783   3/22/2018 0:35:09   
682653  18908914   1139008  51.495785  -2.545169   3/22/2018 0:35:39   

                      city country countryCode  postalCode             state  \
0                      NaN      UK          gb         

In [16]:
# Detect the file encoding
with open("your_local_path/england_global_open_litter_data_material_category.csv", 'rb') as f:
    result = chardet.detect(f.read())
    encoding = result['encoding']

# Read the CSV with the detected encoding
df = pd.read_csv("your_local_path/england_global_open_litter_data_material_category.csv", encoding=encoding)


In [17]:
# Step 1: Function to calculate distance between two coordinates (latitude, longitude)
def calculate_distance(coord1, coord2):
    return geodesic(coord1, coord2).meters

In [18]:
import pandas as pd
import numpy as np
from scipy.spatial import KDTree

# Function to assign categories by location using KDTree
def assign_categories_optimized(df, radius=200):
    # Radius in degrees (approximate conversion for small distances)
    radius_deg = radius / 111000  # 1 degree latitude ≈ 111 km

    # Create a KDTree for efficient spatial queries
    coords = df[['lat', 'long']].values
    tree = KDTree(coords)
    
    # Initialize category_by_location and visited array
    category_by_location = [''] * len(df)
    category_id = 0
    visited = np.zeros(len(df), dtype=bool)

    # Group items by proximity and category
    for i in range(len(df)):
        if visited[i]:
            continue
        
        # Query neighbors within the radius
        indices = tree.query_ball_point(coords[i], radius_deg)
        
        # Group categories
        group_categories = set(df.loc[indices, 'category'])
        
        # Assign category_by_location for each category in the group
        for category in group_categories:
            for idx in indices:
                if df.loc[idx, 'category'] == category:
                    category_by_location[idx] = f"{category_id}-{category}"
        
        # Mark all indices as visited and increment the category ID
        visited[indices] = True
        category_id += 1

    # Add the new column to the DataFrame
    df['category_by_location'] = category_by_location
    return df

# Function to calculate category centers
def calculate_category_centers(df):
    centers = df.groupby('category_by_location').apply(
        lambda group: pd.Series({
            'center_lat': group['lat'].mean(),
            'center_long': group['long'].mean()
        })
    ).reset_index()
    df = pd.merge(df, centers, on='category_by_location', how='left')
    return df

# Apply the optimized functions
df = assign_categories_optimized(df, radius=200)
df = calculate_category_centers(df)

# Save the updated DataFrame to a CSV file
df.to_csv("england_Global_dataset_with_location_categories_centers_optimized.csv", index=False)

# Optional: Print the updated DataFrame
print(df[['litterId', 'lat', 'long', 'category', 'category_by_location', 'center_lat', 'center_long']])


/var/folders/g3/07d8ggrx03ld_fj9md65wkth0000gn/T/ipykernel_16581/2460148727.py:46: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  centers = df.groupby('category_by_location').apply(


        litterId        lat       long                    category  \
0        2480016  52.391107   5.246294              Plastic Litter   
1         577848  26.278606 -81.538590               Uncategorized   
2         577850  26.278606 -81.538590               Uncategorized   
3          88350  50.426128  -5.056561               Uncategorized   
4          88608  52.528403  -2.193209               Uncategorized   
...          ...        ...        ...                         ...   
418949   1093363  -2.545121  51.495582  Paper and Cardboard Litter   
418950   1139003  -2.544095  51.495736      Rubber and Tire Litter   
418951   1139004  -2.544435  51.495776      Rubber and Tire Litter   
418952   1139006  -2.544783  51.495946      Rubber and Tire Litter   
418953   1139008  -2.545169  51.495785                Metal Litter   

                    category_by_location  center_lat  center_long  
0                       0-Plastic Litter   52.391107     5.246294  
1                      